# Data Cleaning: Transactions

In [1]:
import os
from typing import Union

import joblib
import numpy as np
import pandas as pd
import skops.io as sio
from tqdm import tqdm

import helpers
from helpers import (
    CHARTS_DIR, RAW_DATA_DIR, IMPUTER_MODEL_DIR
)

import plotly.express as px
import plotly.graph_objects as go

In [2]:
df_transactions_encoded = pd.read_parquet(RAW_DATA_DIR / 'transactions_KL_ckpt4_encoded.parquet')
df_transactions_encoded

,township_BANDAR BARU SRI PETALING,township_TAMAN TUN DR ISMAIL,township_DAMANSARA HEIGHTS (BUKIT DAMANSARA),township_TAMAN BUKIT MALURI,township_KEPONG BARU,township_OVERSEAS UNION GARDEN,township_HAPPY GARDEN,township_TAMAN MIDAH,township_ALAM DAMAI,township_TAMAN SRI SINAR,...,tenure_FREEHOLD,floors,rooms,land_area,built_up,price_psf,price,year,month,day
0,1,0,0,0,0,0,0,0,0,0,...,0,1.0,NaN,2196.0,NaN,342.0,750000.0,2023,6,9
1,1,0,0,0,0,0,0,0,0,0,...,0,2.0,NaN,753.0,NaN,398.0,300000.0,2023,6,1
2,1,0,0,0,0,0,0,0,0,0,...,0,2.5,NaN,3197.0,NaN,188.0,600000.0,2023,5,29
3,1,0,0,0,0,0,0,0,0,0,...,0,2.0,NaN,753.0,NaN,531.0,400000.0,2023,5,25
4,1,0,0,0,0,0,0,0,0,0,...,0,2.5,NaN,4801.0,NaN,250.0,1200000.0,2023,5,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265265,0,0,0,0,0,0,0,0,0,0,...,0,1.0,2.0,493.0,493.0,71.0,35000.0,1990,11,13
265266,0,0,0,0,0,0,0,0,0,0,...,1,1.0,3.0,1454.0,1454.0,150.0,218025.0,2005,1,10
265267,0,0,0,0,0,0,0,0,0,0,...,1,1.0,3.0,593.0,593.0,194.0,115000.0,2008,2,25
265268,0,0,0,0,0,0,0,0,0,0,...,0,1.0,2.0,1193.0,1193.0,197.0,235000.0,2009,8,10


In [3]:
df_transactions_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265270 entries, 0 to 265269
Columns: 1905 entries, township_BANDAR BARU SRI PETALING to day
dtypes: float64(6), int32(3), int64(1896)
memory usage: 3.8 GB


In [18]:
df_transactions_encoded['year'].value_counts().sort_index()

year
1909       1
1933       1
1955       1
1959       1
1960       7
        ... 
2019    7388
2020    6775
2021    7492
2022    8136
2023    2098
Name: count, Length: 67, dtype: int64

### Concluding Remarks from Data Cleaning 1: Recap
1. The following data cleaning steps has been performed:
    - Removed address column
    - Changed fraction to decimal
    - Removed commas in numerical values
    - Removed units in numerical values
    - Removed exact duplicates
    - Removed outliers using HDBSCAN, based on:
        - Continuous variables: `land_area`, `built_up`, `price_psf` (3D)
        - Ordinal variables: `floors`, `rooms` (2D)
2. Investigated missing values in `built_up` and `rooms`
    - Investigated correlation and association between features to determine which features to use to impute missing values
3. Encoded features for imputation using one hot encoding

Next, we should proceed to impute the missing values in `built_up` and `rooms`.

## Imputing missing values

Based on literature, the following imputation methods have been identified:
1. Random forest imputation (Jager et al., 2021) for MCAR, MAR and MNAR data in various domain
2. Multiple imputation by deterministic regression (Donlen, 2022) for MCAR data in real estate domain
3. MissForest (Waljee et al., 2013) for MCAR data in medical domain
4. Predictive mean matching, PMM (Heidt, 2019) for MAR data in medical domain
5. KNN imputation (Jadhav et al., 2019) for MCAR, MAR and MNAR data in UCI dataset

However, when filtered by domain (real estate), only three methods are identified:
1. Random forest imputation
2. KNN imputation
3. Multiple imputation by deterministic regression

These are machine learning approaches for imputation, where we treat the features with missing values as target variable and the features without missing values as independent variables. In order to obtain a better overview of the performance of the imputation methods, we use cross validation techniques:
1. Split the dataset into train and test, where train are the data with labels and test are the data without labels
2. Split the train dataset into train and validation
3. Cross validate the train data:
    - Create a pipeline with scaler and model
    - Run cross validation with scoring
    - Output both train and validation scores
    - Return the pipeline and cross validation results
4. Train and evaluate the model with validation data
    - Train the pipeline with train data
    - Predict the validation data
    - Evaluate the model with validation data
5. Evaluate the pipeline with validation data and print out the metrics
6. Predict the test data
7. Return the imputed dataset and the fitted pipeline

References:
- Jager et al. (2021): https://www.frontiersin.org/articles/10.3389/fdata.2021.693674/full
- Donlen (2022): https://egrove.olemiss.edu/cgi/viewcontent.cgi?article=3744&context=hon_thesis
- Waljee et al. (2013): https://bmjopen.bmj.com/content/3/8/e002847.citation-tools
- Heidt (2019): https://dc.etsu.edu/cgi/viewcontent.cgi?article=5014&context=etd
- Jadhav et al (2019): https://www.tandfonline.com/doi/full/10.1080/08839514.2019.1637138

In [4]:
from math import sqrt

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import NotFittedError
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    median_absolute_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
)
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import Pipeline

random_state = 42

Model = Union[RandomForestRegressor, KNeighborsRegressor, RandomForestClassifier, KNeighborsClassifier]

#### Imputing `built_up`

The steps are:
1. Remove `rooms` from the dataset as it has too many missing values
2. Cross validate for `built_up` using:
    - Random forest imputation
    - KNN imputation
3. Use the better model to impute `built_up`

In [5]:
target = 'built_up'

# Split the dataset into train and test, where train are the data with labels and test are the data without labels
df_train = df_transactions_encoded[df_transactions_encoded[target].notna()].drop(columns=['rooms']).dropna()
df_test = df_transactions_encoded[df_transactions_encoded[target].isna()].drop(columns=['rooms'])

# Split the train dataset into train and validation
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=[target]), df_train[target], test_size=0.2, random_state=random_state)

In [6]:
def cross_validation_with_pipeline(pipeline: Pipeline, X_train: np.ndarray, y_train: np.ndarray, task: str):

    # Cross validate the train data
    if task == 'regression':
        scoring = ('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error', 'neg_median_absolute_error')
    elif task == 'classification':
        scoring = ('accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'roc_auc')
    else:
        scoring = None

    cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True, n_jobs=4)
    
    return cv_results

##### Cross validation for `built_up` using various techniques

In [7]:
# Create a pipeline with scaler and model
rf_pipeline_built_up = Pipeline([('scaler', StandardScaler()), ('model', RandomForestRegressor(random_state=random_state, n_jobs=4))])

# Cross validate the pipeline
if not os.path.exists(IMPUTER_MODEL_DIR / 'cv_results_rf_built_up.joblib'):
    cv_results_built_up = cross_validation_with_pipeline(rf_pipeline_built_up, X_train, y_train, 'regression')
    joblib.dump(cv_results_built_up, IMPUTER_MODEL_DIR / 'cv_results_rf_built_up.joblib')
else:
    cv_results_built_up = joblib.load(IMPUTER_MODEL_DIR / 'cv_results_rf_built_up.joblib')

pd.DataFrame(cv_results_built_up)

,fit_time,score_time,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error,test_neg_mean_absolute_percentage_error,train_neg_mean_absolute_percentage_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,1221.663679,11.541488,0.824293,0.973771,-357.268651,-136.370194,-4.652190e+15,-1.715361e+15,-0.0,-0.0
1,1200.971122,9.686063,0.798486,0.975722,-379.701082,-131.454077,-5.023980e+15,-1.716289e+15,-0.0,-0.0
2,1233.660341,5.176621,0.854139,0.973795,-316.718636,-137.235538,-2.803023e+15,-1.677708e+15,-0.0,-0.0
3,1236.589329,6.362303,0.844816,0.973615,-323.399088,-138.035930,-5.624051e+15,-1.652899e+15,-0.0,-0.0
4,285.546735,1.645550,0.784114,0.977032,-404.786024,-126.878500,-3.922569e+15,-1.042910e+15,-0.0,-0.0


The cross validation using random forest took 55m35s. The results wasn't that great, with substantial RMSE and high MAPE. But it performed well on median-based metrics: median absolute error.

In [8]:
# Create a pipeline with scaler and model
knn_pipeline_built_up = Pipeline([('scaler', StandardScaler()), ('model', KNeighborsRegressor(n_jobs=4))])

# Cross validate the pipeline
if not os.path.exists(IMPUTER_MODEL_DIR / 'cv_results_knn_built_up.joblib'):
    cv_results_built_up = cross_validation_with_pipeline(knn_pipeline_built_up, X_train, y_train, 'regression')
    joblib.dump(cv_results_built_up, IMPUTER_MODEL_DIR / 'cv_results_knn_built_up.joblib')
else:
    cv_results_built_up = joblib.load(IMPUTER_MODEL_DIR / 'cv_results_knn_built_up.joblib')

pd.DataFrame(cv_results_built_up)

,fit_time,score_time,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error,test_neg_mean_absolute_percentage_error,train_neg_mean_absolute_percentage_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,115.349414,842.074621,0.728349,0.807001,-444.228462,-369.920969,-3.509313e+15,-3.596327e+15,-65.8,-51.4
1,67.173904,835.366826,0.670352,0.815942,-485.639491,-361.949470,-4.940215e+15,-3.815043e+15,-66.6,-51.6
2,115.667840,844.578770,0.728067,0.805778,-432.449172,-373.617202,-4.629208e+15,-4.027341e+15,-65.7,-51.4
3,114.721992,863.657962,0.714709,NaN,-438.488940,NaN,-7.069854e+15,NaN,-66.0,NaN
4,17.214992,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The cross validation using KNN took around 148m4s, and then failed at the fourth fold due to insufficient memory. Looking at the results, KNN performed worst then random forest, with higher RMSE, MAPE and median absolute error.

##### Check model performance on validation data

In [43]:
def validate_model(pipeline: Pipeline, X_train: np.ndarray, y_train: np.ndarray, X_val: np.ndarray, y_val: np.ndarray, task: str):

    try:
        y_val_pred = pipeline.predict(X_val)
    except NotFittedError:
        print('Pipeline not fitted. Start training...')
        pipeline = pipeline.fit(X_train, y_train)
        y_val_pred = pipeline.predict(X_val)

    print("Results for validation set:")
    if task == 'regression':
        print(f"R2 score: {r2_score(y_val, y_val_pred)}")
        print(f"RMSE score: {sqrt(mean_squared_error(y_val, y_val_pred))}")
        print(f"MAPE score: {mean_absolute_percentage_error(y_val, y_val_pred)}")
        print(f"MAE score: {mean_absolute_error(y_val, y_val_pred)}")
        print(f"Median AE score: {median_absolute_error(y_val, y_val_pred)}")
    elif task == 'classification':
        print(f"Accuracy score: {accuracy_score(y_val, y_val_pred)}")
        print(f"Balanced accuracy score: {balanced_accuracy_score(y_val, y_val_pred)}")
        print(f"Macro F1 score: {f1_score(y_val, y_val_pred, average='macro')}")
        print(f"Weighted F1 score: {f1_score(y_val, y_val_pred, average='weighted')}")
        print(f"Macro precision score: {precision_score(y_val, y_val_pred, average='macro')}")
        print(f"Weighted precision score: {precision_score(y_val, y_val_pred, average='weighted')}")
        print(f"Macro recall score: {recall_score(y_val, y_val_pred, average='macro')}")
        print(f"Weighted recall score: {recall_score(y_val, y_val_pred, average='weighted')}")
        print(f"Macro ROC AUC score: {roc_auc_score(y_val, y_val_pred, average='macro', multi_class='ovr')}")
        print(f"Weighted ROC AUC score: {roc_auc_score(y_val, y_val_pred, average='weighted', multi_class='ovr')}")

    return pipeline

In [10]:
model_path = IMPUTER_MODEL_DIR / 'rf_pipeline_built_up.joblib'

if not os.path.exists(model_path):
    rf_pipeline_built_up = validate_model(rf_pipeline_built_up, X_train, y_train, X_val, y_val, 'regression')
    joblib.dump(rf_pipeline_built_up, model_path, compress=('lzma', 9))
else:
    rf_pipeline_built_up = joblib.load(model_path)
    _ = validate_model(rf_pipeline_built_up, X_train, y_train, X_val, y_val, 'regression')

Results for validation set:
R2 score: 0.8080609693884588
RMSE score: 381.9099647033172
MAPE score: 3890573305511410.5
MAE score: 63.87615023388515
Median AE score: 0.0


Random forest took 6m8s for training on training data and prediction of validation data, and 2.5 second for loading fitted model and predicting validation data.

In [15]:
model_path = IMPUTER_MODEL_DIR / 'knn_pipeline_built_up.joblib'

if not os.path.exists(model_path):
    knn_pipeline_built_up = validate_model(knn_pipeline_built_up, X_train, y_train, X_val, y_val, 'regression')
    joblib.dump(knn_pipeline_built_up, model_path, compress=('lzma', 5))
else:
    knn_pipeline_built_up = joblib.load(model_path)
    _ = validate_model(knn_pipeline_built_up, X_train, y_train, X_val, y_val, 'regression')

Results for validation set:
R2 score: 0.6834473458154234
RMSE score: 490.45856274893976
MAPE score: 4185698477203167.0
MAE score: 158.56516414749206
Median AE score: 63.200000000000045


KNN took 4m39s for training on training data and prediction of validation data, and 4m54s for loading fitted model and predicting validation data.

Comparing random forest and KNN on both cross validation and validation data, random forest performed better than KNN.

##### Impute `built_up` using the best model

In [28]:
def impute_with_model(pipeline: Pipeline, df_train: pd.DataFrame, df_test: pd.DataFrame, target: str):

    y_test_pred = pipeline.predict(df_test)

    df_test[target] = y_test_pred
    df_imputed = pd.concat([df_train, df_test])

    return df_imputed

In [29]:
df_test = df_test.drop(columns=[target]).dropna()
df_transactions_built_up_imputed = impute_with_model(rf_pipeline_built_up, df_train, df_test, target='built_up')
df_transactions_built_up_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265270 entries, 252 to 75120
Columns: 1904 entries, township_BANDAR BARU SRI PETALING to day
dtypes: float64(5), int32(3), int64(1896)
memory usage: 3.8 GB


##### Join the imputed `built_up` data with the original data

In [30]:
df_transactions_built_up_imputed = df_transactions_built_up_imputed.join(df_transactions_encoded['rooms'])
df_transactions_built_up_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265270 entries, 252 to 75120
Columns: 1905 entries, township_BANDAR BARU SRI PETALING to rooms
dtypes: float64(6), int32(3), int64(1896)
memory usage: 3.8 GB


In [31]:
# Check for missing values in all columns
for column in df_transactions_built_up_imputed.columns:
    isna_count = df_transactions_built_up_imputed[column].isna().sum()
    if isna_count > 0:
        print(column, isna_count)

rooms 29217


#### Imputing `rooms`

The steps are:
1. Remove `rooms` with less than 5 samples so that CV can be performed
2. Cross validate for `rooms` using:
    - Random forest imputation
    - KNN imputation
3. Use the better model to impute `rooms`

In [32]:
target = 'rooms'

# Split the dataset into train and test, where train are the data with labels and test are the data without labels
df_train = df_transactions_built_up_imputed[df_transactions_built_up_imputed[target].notna()]
df_train = df_train.groupby(target).filter(lambda x : len(x) > 5).dropna() # Drop rooms with less than 5 samples so that CV can be performed
df_test = df_transactions_built_up_imputed[df_transactions_built_up_imputed[target].isna()]

# Split the train dataset into train and validation
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=[target]), df_train[target], test_size=0.2, random_state=random_state)

##### Cross validation for `built_up` using various techniques

In [33]:
# Create a pipeline with scaler and model
rf_pipeline_rooms = Pipeline([('scaler', StandardScaler()), ('model', RandomForestClassifier(random_state=random_state, n_jobs=4))])

# Cross validate the pipeline
if not os.path.exists(IMPUTER_MODEL_DIR / 'cv_results_rf_rooms.joblib'):
    cv_results_rooms = cross_validation_with_pipeline(rf_pipeline_rooms, X_train, y_train, 'regression')
    joblib.dump(cv_results_rooms, IMPUTER_MODEL_DIR / 'cv_results_rf_rooms.joblib')
else:
    cv_results_rooms = joblib.load(IMPUTER_MODEL_DIR / 'cv_results_rf_rooms.joblib')

pd.DataFrame(cv_results_rooms)

d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Repos\GitHub\ti

,fit_time,score_time,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error,test_neg_mean_absolute_percentage_error,train_neg_mean_absolute_percentage_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,28.685828,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,251.693279,10.347663,0.429146,0.996970,-0.734488,-0.052480,-1.645869e+13,-1.788977e+11,-0.0,-0.0
2,51.241544,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,241.388860,6.027060,0.456886,0.997788,-0.696532,-0.045157,-1.788988e+13,-1.192651e+11,-0.0,-0.0
4,232.804780,6.727474,0.506015,0.997310,-0.666395,-0.049760,-1.908254e+13,-2.981628e+10,-0.0,-0.0


The cross validation took 8m5s to complete. There are classes which has only one transactions, therefore the rooms with less than 5 counts were not included for cross validation.

From the cross validation results, the model overfits.

In [34]:
# Create a pipeline with scaler and model
knn_pipeline_rooms = Pipeline([('scaler', StandardScaler()), ('model', KNeighborsClassifier(n_jobs=4))])

# Cross validate the pipeline
if not os.path.exists(IMPUTER_MODEL_DIR / 'cv_results_knn_rooms.joblib'):
    cv_results_rooms = cross_validation_with_pipeline(knn_pipeline_rooms, X_train, y_train, 'regression')
    joblib.dump(cv_results_rooms, IMPUTER_MODEL_DIR / 'cv_results_knn_rooms.joblib')
else:
    cv_results_rooms = joblib.load(IMPUTER_MODEL_DIR / 'cv_results_knn_rooms.joblib')

pd.DataFrame(cv_results_rooms)

d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Repos\GitHub\ti

,fit_time,score_time,test_r2,train_r2,test_neg_root_mean_squared_error,train_neg_root_mean_squared_error,test_neg_mean_absolute_percentage_error,train_neg_mean_absolute_percentage_error,test_neg_median_absolute_error,train_neg_median_absolute_error
0,47.168992,423.701747,0.382646,0.511510,-0.745020,-0.670505,-1.872425e+13,-1.267200e+13,-0.0,-0.0
1,98.414525,406.607987,0.375650,NaN,-0.768132,NaN,-1.443117e+13,NaN,-0.0,NaN
2,51.702898,425.574222,0.360377,0.472983,-0.777078,-0.692177,-1.526604e+13,-1.386457e+13,-0.0,-0.0
3,35.375724,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59.482107,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Check model performance on validation data

In [44]:
model_path = IMPUTER_MODEL_DIR / 'rf_pipeline_rooms.joblib'

if not os.path.exists(model_path):
    rf_pipeline_rooms = validate_model(rf_pipeline_rooms, X_train, y_train, X_val, y_val, 'classification')
    joblib.dump(rf_pipeline_rooms, model_path, compress=('lzma', 9))
else:
    rf_pipeline_rooms = joblib.load(model_path)
    _ = validate_model(rf_pipeline_rooms, X_train, y_train, X_val, y_val, 'classification')

Results for validation set:
Accuracy score: 0.8350705478581416
Balanced accuracy score: 0.33657015622976827
Macro F1 score: 0.376981223121691
Weighted F1 score: 0.831617850387697
Macro precision score: 0.4883131102823608
Weighted precision score: 0.831007007869572
Macro recall score: 0.33657015622976827
Weighted recall score: 0.8350705478581416


d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Repos\GitHub\time-series-house-price-forecasting\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AxisError: axis 1 is out of bounds for array of dimension 1

The pipeline took 2m27s to train and predict. However, the results were not good.

In [ ]:
model_path = IMPUTER_MODEL_DIR / 'knn_pipeline_rooms.joblib'

if not os.path.exists(model_path):
    knn_pipeline_rooms = validate_model(knn_pipeline_rooms, X_train, y_train, X_val, y_val, 'classification')
    joblib.dump(knn_pipeline_rooms, model_path, compress=('lzma', 5))
else:
    knn_pipeline_rooms = joblib.load(model_path)
    _ = validate_model(knn_pipeline_rooms, X_train, y_train, X_val, y_val, 'classification')

Results for validation set:
R2 score: 0.5527093748549827
RMSE score: 0.5689410437602002
MAPE score: 15647437373178.346
Median AE score: 0.0


The pipeline took 2m27s to train and predict. However, the results were not good.

##### Impute `rooms` using the best model

In [ ]:
df_test = df_test.drop(columns=[target]).dropna()
df_transactions_rooms_imputed = impute_with_model(rf_pipeline_rooms, df_train, df_test, target='rooms')
df_transactions_rooms_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 265225 entries, 253 to 76762
Columns: 1905 entries, township_BANDAR BARU SRI PETALING to rooms
dtypes: float64(6), int32(3), int64(1896)
memory usage: 3.8 GB


In [38]:
for column in df_transactions_rooms_imputed.columns:
    isna_count = df_transactions_rooms_imputed[column].isna().sum()
    if isna_count > 0:
        print(column, isna_count)

NameError: name 'df_transactions_rooms_imputed' is not defined